# 0.Import modules

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.options.display.float_format = '{:.4f}'.format
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import pickle
from pathlib import Path


import plotly.express as px
import plotly.graph_objects as go

# 1.Load data

In [3]:
### Paths ###
DATA_PATH = Path("../data")
TRAIN_RAW_PATH = DATA_PATH / "raw/train.csv"
TEST_RAW_PATH = DATA_PATH / "raw/test.csv"
SAMPLE_SUBMISSION_RAW_PATH = DATA_PATH / "raw/sample_submission.csv"

DATA_PROCESSED_PATH = DATA_PATH / "processed"

In [4]:
train = pd.read_csv(TRAIN_RAW_PATH)
test = pd.read_csv(TEST_RAW_PATH)
census = pd.read_csv(DATA_PATH / "raw/census_starter.csv")
sample = pd.read_csv(SAMPLE_SUBMISSION_RAW_PATH)

train_revealed = pd.read_csv(DATA_PATH / "raw/revealed_test.csv")
train = pd.concat([train, train_revealed])

train['ds'] = pd.to_datetime(train['first_day_of_month'])
train['year'] = train['ds'].dt.year
train['month'] = train['ds'].dt.month

# 2.Model

In [8]:
# A bunch of functions

def smape_vector(y_true, y_pred):
    smape = np.zeros(len(y_true))
    
    numinator = np.abs(y_true - y_pred)
    denominator = ((np.abs(y_true) + np.abs(y_pred)) / 2)

    pos_ind = (y_true != 0) | (y_pred != 0)
    smape[pos_ind] = numinator[pos_ind] / denominator[pos_ind]
    
    return 100 * smape


def fix_new_population_in_january(submission):
    COLS = ['GEO_ID','NAME','S0101_C01_026E']
    df2020 = pd.read_csv(DATA_PATH / 'external/census/ACSST5Y2020.S0101-Data.csv',usecols=COLS)
    df2020 = df2020.iloc[1:]
    df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')
    
    df2021 = pd.read_csv(DATA_PATH / 'external/census/ACSST5Y2021.S0101-Data.csv',usecols=COLS)
    df2021 = df2021.iloc[1:]
    df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
    df2021.head()
    
    df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
    adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

    df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
    adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()
    
    submission['adult2020'] = submission.cfips.map(adult2020)
    submission['adult2021'] = submission.cfips.map(adult2021)
    
    submission['microbusiness_density'] = submission['microbusiness_density'] * submission['adult2020'] / submission['adult2021']

In [9]:
for lag in [-1, 1]:
    train[f'microbusiness_density_lag_{lag}'] = train.groupby('cfips')['microbusiness_density'].shift(lag)

In [10]:
train_data = train[(train.first_day_of_month >= '2022-09-01') & (train.first_day_of_month <= '2022-11-01')]

In [11]:
# find the best multiplication for cfips for train_data
mult_column_to_mult = {f'smape_{mult}': mult for mult in [1.00, 1.002, 1.004]}

for mult_column, mult in mult_column_to_mult.items():
    train_data['y_pred'] = train_data['microbusiness_density'] * mult
    train_data[mult_column] = smape_vector(
        y_true=train_data['microbusiness_density_lag_-1'],
        y_pred=train_data['y_pred']
    )
    
df_agg = train_data.groupby('cfips')[list(mult_column_to_mult.keys())].mean()

df_agg['best_mult'] = df_agg.idxmin(axis=1).map(mult_column_to_mult)

cfips_to_best_mult = dict(zip(df_agg.index, df_agg['best_mult']))


In [12]:
df_agg['best_mult'].value_counts(normalize=True)

1.0000   0.4612
1.0040   0.4163
1.0020   0.1225
Name: best_mult, dtype: float64

In [13]:
last_value = train.groupby('cfips', as_index=False)['microbusiness_density'].last().rename(
    columns={'microbusiness_density': 'last_train_value'}
)

submission = test.merge(
    last_value,
    on='cfips'
)

submission['forecast_month_number'] = submission.groupby('cfips').cumcount() + 1

submission['microbusiness_density'] = submission['last_train_value'] * submission['cfips'].map(cfips_to_best_mult)

print(np.mean(submission['microbusiness_density'] == submission['last_train_value']))


0.46124401913875596


In [14]:
last_value.head()

,cfips,last_train_value
0,1001,3.4709
1,1003,8.2506
2,1005,1.2523
3,1007,1.2872
4,1009,1.8521


In [15]:
submission.head()

,row_id,cfips,first_day_of_month,last_train_value,forecast_month_number,microbusiness_density
0,1001_2022-11-01,1001,2022-11-01,3.4709,1,3.4848
1,1001_2022-12-01,1001,2022-12-01,3.4709,2,3.4848
2,1001_2023-01-01,1001,2023-01-01,3.4709,3,3.4848
3,1001_2023-02-01,1001,2023-02-01,3.4709,4,3.4848
4,1001_2023-03-01,1001,2023-03-01,3.4709,5,3.4848


In [24]:
fix_new_population_in_january(submission)

submission[['row_id', 'microbusiness_density']].to_csv('submission.csv', index=False)

In [31]:
submission.head(10)

,row_id,cfips,first_day_of_month,last_train_value,forecast_month_number,microbusiness_density,adult2020,adult2021
0,1001_2022-11-01,1001,2022-11-01,3.4709,1,3.3325,42496,44438
1,1001_2022-12-01,1001,2022-12-01,3.4709,2,3.3325,42496,44438
2,1001_2023-01-01,1001,2023-01-01,3.4709,3,3.3325,42496,44438
3,1001_2023-02-01,1001,2023-02-01,3.4709,4,3.3325,42496,44438
4,1001_2023-03-01,1001,2023-03-01,3.4709,5,3.3325,42496,44438
5,1001_2023-04-01,1001,2023-04-01,3.4709,6,3.3325,42496,44438
6,1001_2023-05-01,1001,2023-05-01,3.4709,7,3.3325,42496,44438
7,1001_2023-06-01,1001,2023-06-01,3.4709,8,3.3325,42496,44438
8,1003_2022-11-01,1003,2022-11-01,8.2506,1,7.9352,171296,178105
9,1003_2022-12-01,1003,2022-12-01,8.2506,2,7.9352,171296,178105
